# 2D Smoothiesem inversion notebook

This notebook goes through every step of the way to perform a synthetic 2D inversion, using Smoothiesem type of meshes.

Unfortunately not all parts of the code can be made publicly available at this time at least so this version of the notebook will not be able to run. It does however show how things are done.

Initially packages are loaded and all used functions defined.
The first iteration is then performed with some communication with Salvus Opt.
After the first iteration is done, then by always naming the new iteration with the same names that Salvus Opt gives it, the rest of the inversion can be performed automatically.

In [ ]:
import os, sys
# sys.path.append('../py_utilities') # Not included in repository

import h5py
import numpy as np
import matplotlib.pyplot as plt
import inspect
import stat
import shutil
import pyasdf
import pathlib
import toml
import math
import obspy
import copy
import pyexodus

import salvus_flow
from salvus_flow import api
# from salvus_flow.sites.site_config import Status
from obspy.signal import filter

from salvus_mesh.structured_grid_2D import StructuredGrid2D
from salvus_mesh.skeleton import Skeleton
import salvus_misfit

# These are from the private py_utilities directory
# from smoothiesem_mesh import smooth_mesh

# from model_handling import attach_homogeneous_model, \
#                            attach_2d_model, \
#                            generate_gll_model, \
#                            generate_2d_model, \
#                            define_region_of_interest

### Create directory structure and save paths

We want a directory tree like this for our projects:

                                                    Inversion
                                                        |
            ------------------------------------------------------------------------------------------------------
            |                |                  |                      |                        |                |
       INFORMATION     ADJOINT_SOURCES      SYNTHETICS               MESHES                    DATA          GRADIENT
            |                |                  |                      |                        |
        ITERATION        ITERATION          ITERATION             -----------                 SOURCE
            |                |                  |                 |         |                   |
          SOURCE          ASDF Files        ASDF Files        ITERATION  True model          True data
            |                                                     |
      ---------------                                         Smoothies
      |             |
    Misfit   Simulation I/O
    
    
#### Define a few classes used and then some functions

In [2]:
# Define a few paths
class Path:
    def __init__(self, name):
        self.root = "/Drive/Users/solvi/workspace/synthetic_inversion/smoothiesem2D/" + name
        self.information = os.path.join(self.root, "INFORMATION")
        self.adjoint_sources = os.path.join(self.root, "ADJOINT_SOURCES")
        self.synthetics = os.path.join(self.root, "SYNTHETICS")
        self.meshes = os.path.join(self.root, "MESHES")
        self.data = os.path.join(self.root, "DATA")
        self.gradients = os.path.join(self.root, "GRADIENTS")
        self.salvusopt = os.path.join(self.root, "SALVUSOPT")
    
    def create_directory_structure(self):
        if os.path.exists(self.root):
            raise Error(f"Path {self.root} already exists")
        # Create a directory structure for project
        os.mkdir(self.root)
        os.mkdir(self.information)
        os.mkdir(self.adjoint_sources)
        os.mkdir(self.synthetics)
        os.mkdir(self.meshes)
        os.mkdir(self.data)
        os.mkdir(self.gradients)
        os.mkdir(self.salvusopt)
        os.mkdir(os.path.join(self.root, "SHELL_RUNS"))
        os.mkdir(os.path.join(self.root, "OUTPUT"))
        print(f"A new project was created in path: {self.root}")

In [3]:
class Parameters:
    def __init__(self, fmax, start_time, end_time, time_step, abs_layers):
        self.fmax = fmax
        self.start_time = start_time
        self.end_time = end_time
        self.time_step = time_step
        self.abs_layers = abs_layers


In [4]:
class Source:
    def __init__(self, name, loc_x, loc_y, scale):
        self.name = name
        self.loc_x = loc_x * 1000  # Input in km but stored in meters.
        self.loc_y = loc_y * 1000
        self.scale = scale
        self.rec_info_dict = {}
    
    def add_receiver(self, network, name, loc_x, loc_y):
        self.rec_info_dict[name] = {"loc_x": loc_x * 1000, "loc_y": loc_y * 1000, "network": network}

In [5]:
def create_project(name):
    # Make an instance of the Path class
    paths = Path(name)
    paths.create_directory_structure()


#### To make a regular grid mesh

In [6]:
def cartesian_mesh(vp, vs, rho, fmax=0.2,
                  elements_per_wavelength=2., minx=0, maxx=0, miny=1000, maxy=1000, plot=False):
    """
    build rectangular mesh
    """
    hmax = vs / fmax / elements_per_wavelength
    nelem_x = int(np.ceil((maxx-minx) / hmax))
    nelem_y = int(np.ceil((maxy-miny) / hmax))
    
    # create box mesh without refinements
    sg = StructuredGrid2D.rectangle(
        nelem_x,nelem_y, min_x=minx, max_x=maxx, min_y=miny, max_y=maxy)

    m = sg.get_unstructured_mesh()

    m.find_side_sets(mode='cartesian')
    attach_homogeneous_model(m, vp, vs, rho)
    
    if plot:
        m.plot()

    return m

#### Input file generation

In [7]:
# Input file generation

def generate_receivers_toml(tomlFileName, info):

    rstr = '[[receiver]]\nnetwork = "RG"\nstation = "%s"\nlocation = ""\n' + \
           'medium = "solid"\n' + \
           '\nsalvus_coordinates = [%f, %f]\n\n'

    with open(tomlFileName, 'w') as f:
        for rec in info:
            f.write(rstr % (rec, info[rec]["loc_x"], info[rec]["loc_y"]))
        
def generate_source_toml(tomlFileName, name, locx, locy, freq, scale):

    sstr = '[[source]]\nname = "%s"\n' + \
           'temporal_type = "ricker"\n' + \
           'spatial_type = "moment_tensor"\n' + \
            'location = [%f, %f]\n' + \
            'center_frequency = %f\n' + \
            'scale = [%f, %f, %f]\n'
            
    with open(tomlFileName, 'w') as f:
        f.write(sstr % (name, locx, locy, freq, scale[0], scale[1], scale[2])) 

def generate_input_files(source, iteration, sim_type, sim_params): # Remember to create iteration first.
    if sim_type == "forward":
        generate_input_files_forward(source, iteration, sim_params)
    elif sim_type == "adjoint":
        generate_input_files_adjoint(source, iteration, sim_params)
    else:
        raise ValueError("Only simulation types forward and adjoint available")

def generate_input_files_forward(source, iteration, sim_params):  
    input_file_direc = os.path.join(paths.information, iteration, source.name, "forward")
    mesh_direc = os.path.join(paths.meshes, iteration, source.name) # Smoothie
#     mesh_direc = os.path.join(paths.meshes, iteration) # Cartesian
    if not os.path.exists(input_file_direc):
        os.mkdir(input_file_direc)
    runfile_name = os.path.join(input_file_direc, "run_salvus_smoothie.sh")
    src_toml = os.path.join(input_file_direc, "source.toml")
    rec_toml =  os.path.join(input_file_direc, "receivers.toml")
    mesh_file = os.path.join(mesh_direc, "smoothiesem_2d.e")
    rec_file_name = os.path.join(input_file_direc, "receivers.h5")
    static_file_name = os.path.join(mesh_direc, "smoothiesem_2d.h5")
    wavefield_file = os.path.join(input_file_direc, "fwd_smoothiesem_2d.h5")
    
    generate_source_toml(tomlFileName=src_toml, name=source.name, locx=source.loc_x, locy=source.loc_y,
                        freq=fmax/2.0, scale=source.scale)
    generate_receivers_toml(tomlFileName=rec_toml, info=source.rec_info_dict)
    
    run_str = MPI_BIN + " -n 16 " + SALVUS_BIN
    run_str += " --dimension 2 " + "--polynomial-order 4 " 
    run_str += "--mesh-file " + mesh_file + " " + "--model-file " 
    run_str += mesh_file + " " + "--load-static-fields model " 
    run_str += "--load-static-file-name " + static_file_name + " " + "--start-time " 
    run_str += str(sim_params.start_time) + " " + "--time-step " + str(sim_params.time_step) + " " 
    run_str += "--end-time " + str(sim_params.end_time) + " " + "--source-toml " + src_toml + " " 
    run_str += "--absorbing-boundaries surface " + "--num-absorbing-layers " + str(sim_params.abs_layers) + " " 
    run_str += "--receiver-toml " + rec_toml + " " + "--receiver-fields u_ELASTIC " 
    run_str += "--receiver-file-name " + rec_file_name + " " + "--save-fields adjoint "
    run_str += "--save-wavefield-file " + wavefield_file + " " + "--io-sampling-rate-volume 20"
    
    with open(runfile_name, "w") as f:
        f.write(run_str)

def generate_input_files_adjoint(source, iteration, sim_params):
    input_file_direc = os.path.join(paths.information, iteration, source.name, "adjoint")
    mesh_direc = os.path.join(paths.meshes, iteration, source.name) # Smoothie
#     mesh_direc = os.path.join(paths.meshes, iteration) # Cartesian
    adjoint_direc = os.path.join(paths.adjoint_sources, iteration, source.name)
    if not os.path.exists(input_file_direc):
        os.mkdir(input_file_direc)
    runfile_name = os.path.join(input_file_direc, "run_salvus_smoothie.sh")
    src_toml = os.path.join(adjoint_direc, "adjoint.toml")
    mesh_file = os.path.join(mesh_direc, "smoothiesem_2d.e")
    static_file_name = os.path.join(mesh_direc, "smoothiesem_2d.h5")
    fwd_wavefield_file = os.path.join(paths.information, iteration, source.name, "forward", "fwd_smoothiesem_2d.h5")
    adj_wavefield_file = os.path.join(input_file_direc, "adj_smoothie.h5")
    kernel_file = os.path.join(input_file_direc, "kernel.e")
    gradient_name = os.path.join(input_file_direc, "gradient_gll.h5")
    
    run_str = MPI_BIN + " -n 16 " + SALVUS_BIN
    run_str += " --dimension 2 " + "--polynomial-order 4 " 
    run_str += "--mesh-file " + mesh_file + " " + "--model-file " 
    run_str += mesh_file + " " + "--load-static-fields model " 
    run_str += "--load-static-file-name " + static_file_name + " " + "--start-time " 
    run_str += str(sim_params.start_time) + " " + "--time-step " + str(sim_params.time_step) + " " 
    run_str += "--end-time " + str(sim_params.end_time) + " " + "--source-toml " + src_toml + " " 
    run_str += "--adjoint " + "--kernel-file " + kernel_file + " --load-fields adjoint "
    run_str += "--load-wavefield-file " + fwd_wavefield_file + " --save-static-fields gradient "
    run_str += "--save-static-file-name " + gradient_name + " --kernel-fields MU,LAMBDA "
    run_str += "--absorbing-boundaries surface " + "--num-absorbing-layers " + str(sim_params.abs_layers) + " "
    run_str += "--save-wavefield-file " + adj_wavefield_file + " --save-fields u_ELASTIC "
    run_str += "--io-sampling-rate-volume 20"
    
    with open(runfile_name, "w") as f:
        f.write(run_str)

def make_gll_model_from_exodus(iteration, source_list=None, mesh_type="smoothiesem"):
    runfile_name = os.path.join(paths.meshes, iteration, "run_salvus_make_gll.sh")
    with open(runfile_name, "w") as f:
        if not source_list:
            source_list = [0]
            if mesh_type == "smoothiesem":
                raise Error("smoothiesem mesh_type needs a list of sources")
        for source in source_list:
            if mesh_type == "cartesian":
                mesh_file = os.path.join(paths.meshes, iteration, "cartesian_2d.e")
                h5model = os.path.join(paths.meshes, iteration, "cartesian_2d.h5")
            if mesh_type == "smoothiesem":
                mesh_file = os.path.join(paths.meshes, iteration, source.name, "smoothiesem_2d.e")
                h5model = os.path.join(paths.meshes, iteration, source.name, "smoothiesem_2d.h5")
            run_str = MPI_BIN + " -n 16 " + SALVUS_BIN
            run_str += f" --mesh-file {mesh_file} --model-file {mesh_file} "
            run_str += f"--dimension 2 " + "--polynomial-order 4 "
            run_str += f"--end-time 1 " + "--save-static-fields model "
            run_str += f"--save-static-file-name {h5model} \n"

            f.write(run_str)
    return runfile_name
    

#### Utility functions

In [8]:

def retrieve_synthetics(source_name, iteration):
    # Find output from salvus and move to the correct directory to prepare for adjoint source calculation
    salvus_output_path = os.path.join(paths.information, iteration, source_name, "forward")
    waveforms = os.path.join(salvus_output_path, "receivers.h5")
    synthetics_path = os.path.join(paths.synthetics, iteration, source_name, "receivers.h5")
    data_path = os.path.join(paths.data, source_name, "receivers.h5")
    shutil.copy(waveforms, synthetics_path)
    
def retrieve_gradient(source_name, iteration):
    # Currently moves both exodus and hdf5 versions of the gradient.
    salvus_adjoint_output_path = os.path.join(paths.information, iteration, source_name, "adjoint")
    gradient = os.path.join(salvus_adjoint_output_path, "kernel.e")
    gradient_gll = os.path.join(salvus_adjoint_output_path, "gradient_gll.h5")
    gradient_gll_xdmf = os.path.join(salvus_adjoint_output_path, "gradient_gll_ELASTIC.xdmf")
    shutil.copy(gradient, os.path.join(paths.gradients, iteration, source_name, "kernel.e"))
    shutil.copy(gradient_gll, os.path.join(paths.gradients, iteration, source_name, "gradient_gll.h5"))
    shutil.copy(gradient_gll_xdmf, os.path.join(paths.gradients, iteration, source_name, "gradient_gll_ELASTIC.xdmf"))

def create_iteration(iteration_name):
    # Create folder structure for iteration so we should never run into problems with directories which 
    # do not exist.
    os.mkdir(os.path.join(paths.gradients, iteration_name))
    os.mkdir(os.path.join(paths.information, iteration_name))
    os.mkdir(os.path.join(paths.synthetics, iteration_name))
    os.mkdir(os.path.join(paths.meshes, iteration_name))
    if not os.path.exists(os.path.join(paths.root, "OUTPUT")):
        os.mkdir(os.path.join(paths.root, "OUTPUT"))
    os.mkdir(os.path.join(paths.root, "OUTPUT", iteration_name))
    os.mkdir(os.path.join(paths.adjoint_sources, iteration_name))
    for src in src_list:
        os.mkdir(os.path.join(paths.gradients, iteration_name, src.name))
        os.mkdir(os.path.join(paths.information, iteration_name, src.name))
        os.mkdir(os.path.join(paths.information, iteration_name, src.name, "forward"))
        os.mkdir(os.path.join(paths.information, iteration_name, src.name, "adjoint"))
        os.mkdir(os.path.join(paths.synthetics, iteration_name, src.name))
        os.mkdir(os.path.join(paths.meshes, iteration_name, src.name))
        os.mkdir(os.path.join(paths.root, "OUTPUT", iteration_name, src.name))
        os.mkdir(os.path.join(paths.adjoint_sources, iteration_name, src.name))

def get_paths_to_waveforms(iteration_name, source_name):
    synthetic = os.path.join(paths.synthetics, iteration_name, source_name, "receivers.h5")
    observed = os.path.join(paths.data, source_name, "receivers.h5")
    return synthetic, observed

def move_meshes_between_iterations(previous_iteration, new_iteration, source_name):
    old_mesh = os.path.join(paths.meshes, previous_iteration , source_name, "smoothiesem_2d.e")
    old_model = os.path.join(paths.meshes, previous_iteration, source_name, "smoothiesem_2d.h5")
    dest_mesh = os.path.join(paths.meshes, new_iteration, source_name, "smoothiesem_2d.e")
    dest_model = os.path.join(paths.meshes, new_iteration, source_name, "smoothiesem_2d.h5")
    shutil.copy(old_mesh, dest_mesh)
    shutil.copy(old_model, dest_model)

def distribute_smoothers(iteration):
    smooth_script = os.path.join(paths.gradients, "smooth_script.sh")
    dest_smoothie = os.path.join(paths.gradients, iteration, "smooth_script.sh")
    shutil.copy(smooth_script, dest_smoothie)
    

#### Adjoint Source Calculations and Misfit Quantification

In [9]:
def calculate_adjoint_sources(iteration, source, sim_params):
    synthetic_waveform, observed_waveform = get_paths_to_waveforms(iteration, source.name)
    
    ad_src_type = "waveform_misfit"
    min_period = 1.0 / sim_params.fmax - 1.0  
    max_period = 1.0 / sim_params.fmax + 1.0
    components = ['XDX', 'XDY']

    with pyasdf.ASDFDataSet(synthetic_waveform, mode="a") as syn:
        with pyasdf.ASDFDataSet(observed_waveform) as obs:
            for receiver in source.rec_info_dict:
                rec = source.rec_info_dict[receiver]["network"] + "_" + receiver
                s=0
                if len(syn.auxiliary_data.list()) != 0:
                    if rec in syn.auxiliary_data["AdjointSources"]:
                        del syn.auxiliary_data["AdjointSources"][rec]
                for component in components:
                    rec = source.rec_info_dict[receiver]["network"] + "_" + receiver
                    env = filter.envelope(obs.waveforms[rec].displacement[s].data)
                    start_window_idx = np.where(np.abs(env) > 0.02 * np.amax(env))[0][0] - 120
                    end_window_idx = np.where(np.abs(env) > 0.02 * np.amax(env))[0][-1] + 120
                    adj_start_time = syn.waveforms[rec].displacement[s].stats.starttime
                    adj_end_time = syn.waveforms[rec].displacement[s].stats.endtime
                    adj_time_step = syn.waveforms[rec].displacement[s].stats.delta
                    start_window = adj_start_time + adj_time_step * start_window_idx - sim_params.start_time
                    end_window = adj_start_time + adj_time_step * end_window_idx - sim_params.start_time
                    window = (start_window, end_window)
                    observed = obspy.Trace(data=obs.waveforms[rec].displacement[s].data, 
                                           header=obs.waveforms[rec].displacement[s].stats)
                    observed = copy.deepcopy(observed)
                    synthetic = obspy.Trace(data=syn.waveforms[rec].displacement[s].data, 
                                            header=syn.waveforms[rec].displacement[s].stats)
                    synthetic = copy.deepcopy(synthetic)
                    adjoint_source = salvus_misfit.calculate_adjoint_source(ad_src_type, observed=observed, 
                                                                        synthetic=synthetic, min_period=min_period, 
                                                                        max_period=max_period, window=window, 
                                                                        plot=False)
                    
                    if adjoint_source.adjoint_source is None:
                        s += 1
                        continue
                    if adjoint_source.adjoint_source is not None:
                        ad_src = adjoint_source.adjoint_source
                        ad_src = ad_src[::-1]
                        misfit =  adjoint_source.misfit
                        path = rec + "/" + component
                        
                        syn.add_auxiliary_data(data=ad_src, data_type="AdjointSources", path=path, 
                                               parameters={"misfit": misfit})
                        s += 1
                    

def write_adjoint_source_to_file(iteration, source, sim_params):
    import toml
    import h5py
    synthetic_waveform, _ = get_paths_to_waveforms(iteration, source.name)
    
    ds = pyasdf.ASDFDataSet(synthetic_waveform)
    adj_srcs = ds.auxiliary_data["AdjointSources"]
    output_dir = os.path.join(paths.adjoint_sources, iteration, source.name)
    receiver_dir = os.path.join(paths.information, iteration, source.name, "forward")
    receivers = toml.load(os.path.join(receiver_dir, "receivers.toml"))["receiver"]
    adjoint_source_file_name = os.path.join(output_dir, "adjoint_source.h5")
    toml_file_name = os.path.join(output_dir, "adjoint.toml")
    
    toml_string = f"source_input_file = \"{adjoint_source_file_name}\"\n\n"
    with h5py.File(adjoint_source_file_name, 'a') as f:
    
        for receiver in adj_srcs.list():
            for channel in adj_srcs[receiver]:
                # Need to do this for what happens later
                x_comp = np.zeros(len(adj_srcs[receiver][channel.path].data.value))
                y_comp = np.zeros(len(adj_srcs[receiver][channel.path].data.value))
                continue
            for channel in adj_srcs[receiver]: 
                if channel.path[-1] == "X":
                    x_comp = adj_srcs[receiver][channel.path].data.value
                elif channel.path[-1] == "Y":
                    y_comp = adj_srcs[receiver][channel.path].data.value
                else:
                    print("Hey! Component neither X nor Y!")
            xy = np.array((x_comp, y_comp)).T
            if receiver in f:
                del f[receiver]
            src = f.create_dataset(receiver, data=xy)
            src.attrs["dt"] = sim_params.time_step
            src.attrs["location"] = np.array([source.rec_info_dict[receiver[-4:]]["loc_x"], 
                                     source.rec_info_dict[receiver[-4:]]["loc_y"]], dtype=np.float64)
            src.attrs["spatial-type"] = np.string_("vector")
            src.attrs['starttime'] = sim_params.start_time * 1.0e9

            toml_string += f"[[source]]\n"
            toml_string += f"name = \"{receiver}\"\n"
            toml_string += f"dataset_name = \"/{receiver}\"\n\n"

    with open(toml_file_name, "w") as fh:
        fh.write(toml_string)
        amount = len(adj_srcs.list())
        print(f"Wrote {amount} adjoint sources to file")
        
        
def get_misfit_for_source(iteration, source):
    
    synthetic_waveform, _ = get_paths_to_waveforms(iteration, source.name)
    
    with pyasdf.ASDFDataSet(synthetic_waveform, mode="r") as ds:
        adj_src_data = ds.auxiliary_data["AdjointSources"]
        stations = ds.auxiliary_data["AdjointSources"].list()
        total_misfit = 0.0
        for station in stations:
            channels = adj_src_data[station].list()
            for channel in channels:
                misfit = adj_src_data[station][channel].parameters["misfit"]
                total_misfit += misfit
                
    return total_misfit
    
        
def write_misfit_to_file(iteration, src_list):
    import toml
    misfit = 0.0
    toml_dict = {}
    toml_dict["misfits"] = {}
    toml_dict["misfits"]["source"] = {}
    toml_dict["misfits"]["total"] = {}
    toml_file_name = os.path.join(paths.adjoint_sources, iteration, "misfits.toml")

    for src in src_list:
        print(src.name)
        event_misfit = get_misfit_for_source(iteration, src)
        toml_dict["misfits"]["source"][src.name] = event_misfit
        misfit += event_misfit
    toml_dict["misfits"]["total"]["total_misfit"] = misfit
    
    with open(toml_file_name, "w") as fh:
        toml.dump(toml_dict, fh)
        

#### Preparation of shell scripts to run salvus or smoother

In [10]:
def create_runfile(iteration_name, source_list, sim_type):
    file_name = "run_" + sim_type + "_simulations_" + iteration_name + ".sh"
    file = os.path.join(paths.root, "SHELL_RUNS", file_name)
    with open(file, "w") as fh:
        for source in source_list:
            run_file = os.path.join(paths.information, iteration_name, source.name, sim_type, "run_salvus_smoothie.sh")
            outfile = os.path.join(paths.root, "OUTPUT", iteration_name, source.name, sim_type + ".txt")
            fh.write("sh " + run_file + " | tee " + outfile + " \n")
    run_command = "sh " + file
    return run_command

def run_salvus_opt():
    file_name = "run_salvus_opt.sh"
    file = os.path.join(paths.root, "SHELL_RUNS", file_name)
    with open(file, "w") as fh:
        run_file = os.path.join(paths.salvusopt, "run_salvus_opt.sh")
        fh.write("cd " + paths.salvusopt + " \n")
        fh.write("sh " + run_file + "\n")
        fh.write("cd -")
    run_command = "sh " + file
    return run_command

def smooth_gradient(iteration):
    file_name = "run_smoother_" + iteration + ".sh"
    file = os.path.join(paths.root, "SHELL_RUNS", file_name)
    with open(file, "w") as fh:
        
            smooth_script = os.path.join(paths.gradients, iteration, "smooth_script.sh")
            grad_dir = os.path.join(paths.gradients, iteration)
            fh.write(f"cd {grad_dir} \n")
            if not os.path.exists(os.path.join(grad_dir, "OUT")):
                fh.write("mkdir OUT \n")
            fh.write("sh " + smooth_script + "\n")
    run_command = "sh " + file
    return run_command

#### Interpolation functions

In [11]:
import multi_mesh.api as mapi

def interpolate_from_cartesian_to_smoothie(iteration, src_name):
    cartesian = os.path.join(paths.meshes, iteration, "cartesian_2d.h5")
    smoothie = os.path.join(paths.meshes, iteration, src_name, "smoothiesem_2d.h5")
    print(f"Interpolating velocities for iteration: {iteration}, source: {src_name}")
    mapi.gll_2_gll(cartesian=cartesian, smoothie=smoothie)

def interpolate_from_cartesian_to_smoothie_exodus(iteration, src_name):
    cartesian = os.path.join(paths.meshes, iteration, "cartesian_2d.h5")
    smoothie = os.path.join(paths.meshes, iteration, src_name, "smoothiesem_2d.e")
    print(f"Interpolating velocities for iteration: {iteration}, source: {src_name}")
    mapi.gll_2_exodus(cartesian=cartesian, smoothie=smoothie)

def interpolate_gradient_to_cartesian(iteration, src_list):
    i = 0
    cartesian = os.path.join(paths.gradients, iteration, "cartesian_kernel.e")
    for src in src_list:
        print(f"Interpolating gradient from source: {src.name}")
        smoothie_kern = os.path.join(paths.gradients, iteration, src.name, "gradient_gll.h5")
        if i == 0:
            mapi.gradient_2_cartesian_hdf5(gradient=smoothie_kern, cartesian=cartesian, first=True)
        else:
            mapi.gradient_2_cartesian_hdf5(gradient=smoothie_kern, cartesian=cartesian, first=False)
        i += 1
        
def interpolate_smoothie_to_cartesian_hdf5(iteration, src_list):
    i = 0
    cartesian = os.path.join(paths.gradients, iteration, "cartesian_kernel.h5")
    for src in src_list:
        print(f"Interpolating gradient from source: {src.name}")
        smoothie_kern = os.path.join(paths.gradients, iteration, src.name, "gradient_gll.h5")
        if i == 0:
            mapi.gll_2_gll_gradients(smoothie_kern, cartesian, first=True)
        else:
            mapi.gll_2_gll_gradients(smoothie_kern, cartesian, first=False)
        i += 1

        

    

#### Cut source and receivers from gradients

In [12]:
from scipy.spatial import cKDTree
def cut_receiver_from_gradient(source, iteration, radius_in_km):
    # Cut a small area around the receiver locations from the gradient
    # Loop through receivers that record the source and find which gll points are close
    # put gradient values to zero on these points.
    gradient = os.path.join(paths.gradients, iteration, source.name, "gradient_gll.h5")
    orig = os.path.join(paths.gradients, iteration, source.name, "original_gradient.h5")
    if not os.path.exists(orig):
        shutil.copy(gradient, orig)
    else:
        shutil.copy(orig, gradient)
    a = 0
    receivers = np.zeros([len(source.rec_info_dict),2])
    for rec in source.rec_info_dict:
        rec_x = source.rec_info_dict[rec]["loc_x"]
        rec_y = source.rec_info_dict[rec]["loc_y"]
        
        receivers[a] = [rec_x, rec_y]
        a += 1

    points_to_search = 100
    with h5py.File(gradient, 'r+') as grad:
        grad_points = grad['ELASTIC/coordinates'][:]
        grad_data = grad['ELASTIC/data'][:]
    
        gll_points = 25
        for receiv in receivers:
            for i in range(gll_points):
                point_tree = cKDTree(grad_points[:, i, :])

                idx = point_tree.query_ball_point(receiv, radius_in_km * 1000.0)
                grad_data[0,idx,:,i] = 0.0
                grad['ELASTIC/data'][:] = grad_data

def cut_source_from_gradient(source, iteration, radius_in_km, copy=False):
    # Cut the source region from gradient.
    # Basically the same as the receiver cutting function except now we use a larger radius.
    gradient = os.path.join(paths.gradients, iteration, source.name, "gradient_gll.h5")
    orig = os.path.join(paths.gradients, iteration, source.name, "original_gradient.h5")
    if copy:
        if not os.path.exists(orig):
            shutil.copy(gradient, orig)
        else:
            shutil.copy(orig, gradient)
    src = [source.loc_x, source.loc_y]
    points_to_search = 16
    with h5py.File(gradient, 'r+') as grad:
        grad_points = grad['ELASTIC/coordinates'][:]
        grad_data = grad['ELASTIC/data'][:]
    
        gll_points = 25
        for i in range(gll_points):
            point_tree = cKDTree(grad_points[:, i, :])
            idx = point_tree.query_ball_point(src, radius_in_km * 1000.0)
            grad_data[0,idx,:,i] = 0.0
        grad['ELASTIC/data'][:] = grad_data

def cut_stuff_from_gradient(location, iteration, radius_in_km):
    gradient = os.path.join(paths.gradients, iteration, source.name, "gradient_gll.h5")
    orig = os.path.join(paths.gradients, iteration, source.name, "original_gradient.h5")
    if not os.path.exists(orig):
        shutil.copy(gradient, orig)
    else:
        shutil.copy(orig, gradient)
    src = location
    print(src)
    points_to_search = 16
    with h5py.File(gradient, 'r+') as grad:
        grad_points = grad['ELASTIC/coordinates'][:]
        grad_data = grad['ELASTIC/data'][:]
    
        gll_points = 25
        for i in range(gll_points):
            point_tree = cKDTree(grad_points[:, i, :])
            idx = point_tree.query_ball_point(src, radius_in_km * 1000.0)
            grad_data[0,idx,:,i] = 0.0
        grad['ELASTIC/data'][:] = grad_data

#### Salvus Opt Communication

In [13]:
import toml
def move_gradient_for_salvus_opt(iteration):
    salvusopt_dir = os.path.join(paths.root, "SALVUSOPT")
    if not os.path.exists(salvusopt_dir):
        os.mkdir(salvusopt_dir)
    grad_name = "gradient_" + iteration + ".e"
    dest_gradient = os.path.join(salvusopt_dir, "PHYSICAL_MODELS", grad_name)
    start_gradient = os.path.join(paths.gradients, iteration, "smooth_model.e")
    shutil.copy(start_gradient, dest_gradient)

def get_model_from_salvus_opt(iteration):
    salvusopt_dir = os.path.join(paths.root, "SALVUSOPT")
    if not os.path.exists(salvusopt_dir):
        os.mkdir(salvusopt_dir)
    model_name = iteration + ".e"
    dest_model = os.path.join(paths.meshes, iteration, "cartesian_2d.e")
    start_model = os.path.join(salvusopt_dir, "PHYSICAL_MODELS", model_name)
    shutil.copy(start_model, dest_model)

def write_in_gradient_path_to_task_toml(iteration):
    grad_name = os.path.join(paths.salvusopt, "PHYSICAL_MODELS", "gradient_" + iteration + ".e")
    toml_filename = os.path.join(paths.salvusopt, "task.toml")
    task_dict = toml.load(toml_filename)
    task_dict["task"][0]["output"]["gradient"] = grad_name
    with open(toml_filename, "w") as fh:
        toml.dump(task_dict, fh)
    
def write_misfit_to_task_toml(iteration):
    misfits_toml = os.path.join(paths.adjoint_sources, iteration, "misfits.toml")
    misfits_dict = toml.load(misfits_toml)
    toml_filename = os.path.join(paths.salvusopt, "task.toml")
    misfit = misfits_dict["misfits"]["total"]["total_misfit"]
    task_dict = toml.load(toml_filename)
    task_dict["task"][0]["output"]["misfit"] = float(misfit)
    with open(toml_filename, "w") as fh:
        toml.dump(task_dict, fh)

def close_salvus_opt_task():
    toml_filename = os.path.join(paths.salvusopt, "task.toml")
    task_dict = toml.load(toml_filename)
    task_dict["task"][0]["status"]["open"] = False
    with open(toml_filename, "w") as fh:
        toml.dump(task_dict, fh)

def get_new_iteration_names(iteration):
    toml_filename = os.path.join(paths.salvusopt, "task.toml")
    task_dict = toml.load(toml_filename)
    new_iter = task_dict["task"][0]["input"]["model"].split("/")[-1][:-2]
    prev_iter = iteration
    return prev_iter, new_iter

def read_salvus_opt_task():
    toml_filename = os.path.join(paths.salvusopt, "task.toml")
    task_dict = toml.load(toml_filename)
    task = task_dict["task"][0]["type"]
    return task

## Now lets create the project

This should of course be commented out if project already exists

In [14]:
project_name = "inversion_0"
create_project(project_name)

NameError: name 'Error' is not defined

#### Make Notebook know in which project we want to work

In [15]:
paths = Path(project_name)

#### Define a few simulations parameters
Central frequency, starting time, ending time, time step

In [16]:
sim_params = Parameters(1/5.0, -32.0, 250.0, 0.05, 0.5)

### Add sources and receivers

This can be done arbitrarily but be careful that the sources and receivers are not too close to the boundary so that the absorbing boundaries will not be problematic. Make sure to have the parameter src_list correct as it is used multiple times in the code. It should include the actual source objects, not just the names.

In [17]:
# Source

src_00 = Source("src_00", 400, 400, [1e18, 5e17, 1e16])
src_01 = Source("src_01", 400, 700, [5e17, 1e18, 1e17])
src_02 = Source("src_02", 400, 1000, [1e17, 1e17, 1e18])
src_10 = Source("src_10", 700, 400, [1e18, 1e18, 1e18])
src_11 = Source("src_11", 700, 700, [1e18, 1e18, 1e17])
src_12 = Source("src_12", 700, 1000, [5e17, 5e17, 5e17])
src_20 = Source("src_20", 1000, 400, [1e18, 1e17, 1e17])
src_21 = Source("src_21", 1000, 700, [1e17, 5e17, 1e18])
src_22 = Source("src_22", 1000, 1000, [1e18, 4e17, 4e17])

src_list = [src_00, src_01, src_02, src_10, src_11, src_12, src_20, src_21, src_22]


rec_loc_x = np.arange(300, 1200, 100)
rec_loc_y = np.arange(300, 1200, 100)

src_x = []
src_y = []
for src in src_list:

    b = 0
    for s in rec_loc_x:
        if b < 10:
            x_name = "0" + str(b)
        else:
            x_name = str(b)
        i = 0
        for k in rec_loc_y:
            if np.sqrt((k - src.loc_y / 1000.0) ** 2 + (s - src.loc_x / 1000.0) ** 2) < 100:
                i += 1
                continue
            if i < 10:
                y_name = "0" + str(i)
            else:
                y_name = str(i)
            name = x_name + y_name
            src.add_receiver("RG", name, s, k)
            i += 1
        b += 1
    src_x.append(src.loc_x/1000)
    src_y.append(src.loc_y/1000)


### Mesh generation

Unfortunately this code can not be made public at this point in time

In [19]:
# MPI_BIN="/Drive/Shared/Salvus/bin/mpirun"
MPI_BIN="/home/solvi/miniconda3/envs/lasif/bin/mpirun"
SALVUS_BIN="/Drive/Shared/Salvus/bin/SalvusCompute"

### Create a new iteration

From here we can start the first iteration. Create a new iteration and go from there.

Once a new iteration is created we move the homogeneous smoothiesems to their respective directories. We interpolate the correct velocities on to them and then we are ready for forward simulations.

We can then generate the correct input files for Salvus, we should not need any new receiver.toml we can just copy the previously existing files.

We then run the forward simulations, create the adjoint sources, run the adjoint simulations and interpolate the back on to the cartesian master model.

The model is then updated and we start the looping process.

In [21]:
iteration_name = "it0000" 
create_iteration(iteration_name)

Move the generated meshes into the first iteration and create hdf5 versions of them.

In [22]:
for src in src_list:
    found_mesh = os.path.join(paths.meshes, "INITIAL", src.name, "smoothiesem_2d.e")
    dest_mesh = os.path.join(paths.meshes, iteration_name, src.name, "smoothiesem_2d.e")
    shutil.copy(found_mesh, dest_mesh)
run_file = make_gll_model_from_exodus(iteration_name, source_list=src_list, mesh_type="smoothiesem")
# run_file = make_gll_model_from_exodus(iteration_name, source_list=src_list, mesh_type="cartesian")


In [23]:
%%time
! sh $run_file


Salvus version 0.0.1-1305-g80e675d9
Floating point size: 32
Compiled for GLL orders: 4, 1
Running on 16 rank(s).

--> Server: 'https://l.mondaic.com/licensing_server', User: 'soelvi', Group: 'SWP_Test'.
--> Negotiating 16 license instance(s) for 'SalvusWaveAlpha' [license version 0.0.1] for 3600 seconds ...
--> Success! [Total duration: 0.32 seconds]
Job started Tue Mar  5 10:38:24 2019

No source specified. Set sources with either --cmd-line-source or --source-toml.
Initializing problem.
Time step set to: 0.175804 s.
Begin time loop.
Time loop completed in 0.000611776 seconds.
Begin post processing.
Problem complete.

Job finished Tue Mar  5 10:38:24 2019


--> Releasing license early ...
--> Success! [Total duration: 0.57 seconds]

Salvus version 0.0.1-1305-g80e675d9
Floating point size: 32
Compiled for GLL orders: 4, 1
Running on 16 rank(s).

--> Server: 'https://l.mondaic.com/licensing_server', User: 'soelvi', Group: 'SWP_Test'.
--> Negotiating 16 license instance(s) for 'SalvusWa

### Generate input files for new iteration

Generate the files that salvus needs in order to run a forward simulation on the smoothiesem meshes.

In [24]:
for source in src_list:
    generate_input_files(source=source, iteration=iteration_name, sim_params=sim_params, sim_type="forward")
    print(source.name)


src_00
src_01
src_02
src_10
src_11
src_12
src_20
src_21
src_22


#### Run a forward simulations for all the sources.

In [25]:
run_command = create_runfile(iteration_name, src_list, "forward")

In [26]:
! $run_command


Salvus version 0.0.1-1305-g80e675d9
Floating point size: 32
Compiled for GLL orders: 4, 1
Running on 16 rank(s).

--> Server: 'https://l.mondaic.com/licensing_server', User: 'soelvi', Group: 'SWP_Test'.
--> Negotiating 16 license instance(s) for 'SalvusWaveAlpha' [license version 0.0.1] for 3600 seconds ...
--> Success! [Total duration: 0.87 seconds]
Job started Thu Feb 28 09:50:08 2019

Less than 7 damping boundary layers specified. This may produce stronger boundary reflections than simple Clayton-Enquist type boundaries. We have found that 7+ or 8+ layers damps reflections quite well. Proceed with caution.
Initializing problem.
Timestep set by user as: 0.05 s.
Automated timestep would be set to: 0.0332624 s.
Begin time loop.
Time loop progress...                                       [99.29%]Time loop completed in 0.833464 seconds.
Begin post processing.
Problem complete.

Job finished Thu Feb 28 09:50:10 2019


--> Releasing license early ...
--> Success! [Total duration: 1.07 sec

--> Success! [Total duration: 0.54 seconds]
Job started Thu Feb 28 09:50:25 2019

Less than 7 damping boundary layers specified. This may produce stronger boundary reflections than simple Clayton-Enquist type boundaries. We have found that 7+ or 8+ layers damps reflections quite well. Proceed with caution.
Initializing problem.
Timestep set by user as: 0.05 s.
Automated timestep would be set to: 0.0332621 s.
Begin time loop.
Time loop progress...                                       [99.29%]Time loop completed in 1.02306 seconds.
Begin post processing.
Problem complete.

Job finished Thu Feb 28 09:50:27 2019


--> Releasing license early ...
--> Success! [Total duration: 0.53 seconds]

Salvus version 0.0.1-1305-g80e675d9
Floating point size: 32
Compiled for GLL orders: 4, 1
Running on 16 rank(s).

--> Server: 'https://l.mondaic.com/licensing_server', User: 'soelvi', Group: 'SWP_Test'.
--> Negotiating 16 license instance(s) for 'SalvusWaveAlpha' [license version 0.0.1] for 3600 seconds

### Put synthetics in its correct place

Get it ready for adjoint source calculations.

In [27]:
for src in src_list:
    retrieve_synthetics(source_name=src.name, iteration=iteration_name)

### Calculate adjoint sources at receiver positions

In [30]:
%%time
m = 0

for src in src_list:
    print(src.name)
    
    calculate_adjoint_sources(iteration=iteration_name, source=src, sim_params=sim_params)
    
    write_adjoint_source_to_file(iteration=iteration_name, source=src, sim_params=sim_params)
    m += 1


src_00


/home/solvi/miniconda3/envs/lasif/lib/python3.6/site-packages/numpy/core/numerictypes.py:743: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  FutureWarning, stacklevel=2


Wrote 80 adjoint sources to file
src_01
Wrote 80 adjoint sources to file
src_02
Wrote 80 adjoint sources to file
src_10
Wrote 80 adjoint sources to file
src_11
Wrote 80 adjoint sources to file
src_12
Wrote 80 adjoint sources to file
src_20
Wrote 80 adjoint sources to file
src_21
Wrote 80 adjoint sources to file
src_22
Wrote 80 adjoint sources to file
CPU times: user 1min 27s, sys: 468 ms, total: 1min 28s
Wall time: 1min 28s


#### Write misfits to file so salvus opt can use them later

In [31]:
write_misfit_to_file(iteration_name, src_list)

src_00
src_01
src_02
src_10
src_11
src_12
src_20
src_21
src_22


#### Generate input files for adjoint simulations

In [33]:
for src in src_list:
    generate_input_files(source=src, iteration=iteration_name, sim_params=sim_params, sim_type="adjoint")

#### Run adjoint simulations

In [34]:
run_command = create_runfile(iteration_name, src_list, "adjoint")


In [35]:
! $run_command


Salvus version 0.0.1-1305-g80e675d9
Floating point size: 32
Compiled for GLL orders: 4, 1
Running on 16 rank(s).

--> Server: 'https://l.mondaic.com/licensing_server', User: 'soelvi', Group: 'SWP_Test'.
--> Negotiating 16 license instance(s) for 'SalvusWaveAlpha' [license version 0.0.1] for 3600 seconds ...
--> Success! [Total duration: 0.68 seconds]
Job started Thu Feb 28 09:53:56 2019

Less than 7 damping boundary layers specified. This may produce stronger boundary reflections than simple Clayton-Enquist type boundaries. We have found that 7+ or 8+ layers damps reflections quite well. Proceed with caution.
Initializing problem.
Timestep set by user as: 0.05 s.
Automated timestep would be set to: 0.0332624 s.
Begin time loop.
Time loop progress...                                       [99.29%]Time loop completed in 0.809093 seconds.
Begin post processing.
Problem complete.

Job finished Thu Feb 28 09:53:57 2019


--> Releasing license early ...
--> Success! [Total duration: 0.55 sec

--> Success! [Total duration: 0.39 seconds]
Job started Thu Feb 28 09:54:09 2019

Less than 7 damping boundary layers specified. This may produce stronger boundary reflections than simple Clayton-Enquist type boundaries. We have found that 7+ or 8+ layers damps reflections quite well. Proceed with caution.
Initializing problem.
Timestep set by user as: 0.05 s.
Automated timestep would be set to: 0.0332621 s.
Begin time loop.
Time loop progress...                                       [99.29%]Time loop completed in 0.723154 seconds.
Begin post processing.
Problem complete.

Job finished Thu Feb 28 09:54:10 2019


--> Releasing license early ...
--> Success! [Total duration: 0.42 seconds]

Salvus version 0.0.1-1305-g80e675d9
Floating point size: 32
Compiled for GLL orders: 4, 1
Running on 16 rank(s).

--> Server: 'https://l.mondaic.com/licensing_server', User: 'soelvi', Group: 'SWP_Test'.
--> Negotiating 16 license instance(s) for 'SalvusWaveAlpha' [license version 0.0.1] for 3600 second

#### Move Gradients to the correct folder

In [36]:
for source in src_list:
    retrieve_gradient(source.name, iteration_name)

#### Make an empty cartesian mesh and sum gradients on top of it.

In [38]:
cart_mesh = os.path.join(paths.gradients, "kernel.e")
cart_dest = os.path.join(paths.gradients, iteration_name, "cartesian_kernel.e")
shutil.copy(cart_mesh, cart_dest)

for source in src_list:
    cut_receiver_from_gradient(source, iteration_name, 5.0)
    cut_source_from_gradient(source, iteration_name, 30.0, copy=False)
interpolate_gradient_to_cartesian(iteration_name, src_list)

Interpolating gradient from source: src_00
['LAMBDA', 'MU']
LAMBDA
MU
Interpolating gradient from source: src_01
['LAMBDA', 'MU']


/home/solvi/miniconda3/envs/lasif/lib/python3.6/site-packages/pyasdf/asdf_data_set.py:52: UserWarning: Could not find an element which this points fits into. Maybe you should add some tolerance. Will return the first searched element
  closure_warn(self, *args, **kwargs)


LAMBDA
MU
Interpolating gradient from source: src_02
['LAMBDA', 'MU']
LAMBDA
MU
Interpolating gradient from source: src_10
['LAMBDA', 'MU']
LAMBDA
MU
Interpolating gradient from source: src_11
['LAMBDA', 'MU']
LAMBDA
MU
Interpolating gradient from source: src_12
['LAMBDA', 'MU']
LAMBDA
MU
Interpolating gradient from source: src_20
['LAMBDA', 'MU']
LAMBDA
MU
Interpolating gradient from source: src_21
['LAMBDA', 'MU']
LAMBDA
MU
Interpolating gradient from source: src_22
['LAMBDA', 'MU']
LAMBDA
MU


#### Smooth summed gradient

In [39]:
distribute_smoothers(iteration_name)
run_smoother = smooth_gradient(iteration_name)

In [40]:
! $run_smoother


Salvus version 0.0.1-996-g1752c66
Floating point size: 32
Compiled for GLL orders: 1
Running on 16 rank(s).

No source specified. Set sources with either --cmd-line-source or --source-toml.
Initializing problem.
Timestep set by user as: 500000 s.
Automated timestep would be set to: 1 s.
Begin time loop.
Time loop progress [100%].
Time loop completed in 0.0449409 seconds.
Begin post processing.
Problem complete.


Salvus version 0.0.1-996-g1752c66
Floating point size: 32
Compiled for GLL orders: 1
Running on 16 rank(s).

No source specified. Set sources with either --cmd-line-source or --source-toml.
Initializing problem.
Timestep set by user as: 500000 s.
Automated timestep would be set to: 1 s.
Begin time loop.
Time loop progress [100%].
Time loop completed in 0.060892 seconds.
Begin post processing.
Problem complete.



In [41]:
move_gradient_for_salvus_opt(iteration_name)

In [42]:
write_in_gradient_path_to_task_toml(iteration_name)
write_misfit_to_task_toml(iteration_name)
close_salvus_opt_task()
run_file = run_salvus_opt()

In [43]:
! $run_file


Salvus (c) The Salvus Development Team. 2018

SalvusOpt version: 921e8ab
SalvusUtility version: 0989385

Third party libraries
PETSc version: 3.8.4.0 (git hash v3.8.4)
HDF5 library version: 1.10.2
Netcdf version: 4.6.1
ExodusII version: 7.020000

Floating point size: 32
Integer size: 32
Running on 1 rank(s).

--> Server: 'https://l.mondaic.com/licensing_server', User: 'soelvi', Group: 'SWP_Test'.
--> Negotiating 1 license instance(s) for 'SalvusOptAlpha' [license version 0.0.1] for 3600 seconds ...
--> Success! [Total duration: 0.41 seconds]
Job started Thu Feb 28 09:57:16 2019

method name = lbfgs
method type = trust-region
method mode = interactive
Inversion parameter found: MU
Inversion parameter found: LAMBDA
Read initial model from /mnt/md0/Drive/Users/solvi/workspace/synthetic_inversion/smoothiesem2D/nlat02/MESHES/INITIAL/cartesian_2d.e
No bounds detected. Solving unconstrained problem.
mType trust-region
mMode interactive
mTomlDir /Drive/Users/solvi/workspace/synthetic_inversio

## Looping region

The shell below can perform an arbitrary number of iterations automatically

In [46]:
%%time
import time
max_it = 30
rejected = 0
simulation_time = 0
adjoint_source_time = 0
interpolation_time = 0
cutting_time = 0
for i in range(max_it):
    print("==================================================================== \n")
    print(str(i) + "\n")
    print("==================================================================== \n")
    
    # Name of an iteration which has the correct shape of meshes
    iteration = "it0000"
    task = read_salvus_opt_task()
    if task != "compute_misfit":
        print("SOMETHING WRONG")
        break
    previous_iteration, iteration = get_new_iteration_names(iteration)
    # # Create Iteration
    
    create_iteration(iteration)
    
    print("==================================================================== \n \n")
    print(f"Current iteration: {iteration} \n \n")
    print("==================================================================== \n")
    get_model_from_salvus_opt(iteration)
    run_file = make_gll_model_from_exodus(iteration, mesh_type="cartesian")
    ! sh $run_file

    # Copy meshes to the correct places
    for src in src_list:
        move_meshes_between_iterations(previous_iteration, iteration, src.name)

    # Interpolate velocities to the meshes
    start = time.time()
    for src in src_list:
        interpolate_from_cartesian_to_smoothie(iteration, src.name)
    end = time.time()
    interpolation_time += (end - start)

    # Generate input files forward
    for src in src_list:
        generate_input_files(src, iteration, "forward", sim_params)

    # Run forward simulations
    start = time.time()
    run_command = create_runfile(iteration, src_list, "forward")
    ! $run_command
    end = time.time()
    simulation_time += (end - start)

    # Retrieve synthetics
    print(f"Iteration: {iteration}")
    for src in src_list:
        retrieve_synthetics(src.name, iteration)

    # Generate Adjoint sources
    start = time.time()
    for src in src_list:
        print(f"Calculating adjoint sources for: {src.name}")
        calculate_adjoint_sources(iteration=iteration, source=src, sim_params=sim_params)
        # Write them to file
        write_adjoint_source_to_file(iteration=iteration, source=src, sim_params=sim_params)  
    end = time.time()
    adjoint_source_time += (end - start)
    write_misfit_to_file(iteration, src_list)


    write_misfit_to_task_toml(iteration)
    close_salvus_opt_task()
    run_file = run_salvus_opt()    

    ! $run_file

    task = read_salvus_opt_task()
    if task != "compute_gradient":
        print("==================================================================== \n \n")
        print("                         MODEL REJECTED \n \n")
        print("==================================================================== \n")
        rejected += 1
        continue
    # Generate input files adjoint
    for src in src_list:
        generate_input_files(src, iteration, "adjoint", sim_params)

    # Run adjoint simulations
    start = time.time()
    run_adjoint = create_runfile(iteration, src_list, "adjoint")
    ! $run_adjoint
    end = time.time()
    simulation_time += (end - start)

    # Retrieve gradients
    for src in src_list:
        retrieve_gradient(src.name, iteration)
    cart_mesh = os.path.join(paths.gradients, "kernel.e")
    cart_dest = os.path.join(paths.gradients, iteration, "cartesian_kernel.e")
    shutil.copy(cart_mesh, cart_dest)

    start = time.time()
    for source in src_list:
        cut_receiver_from_gradient(source, iteration, 5.0)
        cut_source_from_gradient(source, iteration, 30.0, copy=False)
    end = time.time()
    cutting_time += (end - start)
    
    # sum all the smoothie source gradients on top of it
    start = time.time()
    interpolate_gradient_to_cartesian(iteration, src_list)
    end = time.time()
    interpolation_time += (end - start)
    
    # Smooth gradients
    distribute_smoothers(iteration)
    run_smoother = smooth_gradient(iteration)
    ! $run_smoother

    move_gradient_for_salvus_opt(iteration)

    write_in_gradient_path_to_task_toml(iteration)
    close_salvus_opt_task()
    run_file = run_salvus_opt()
    ! $run_file

    close_salvus_opt_task()
    run_file = run_salvus_opt()
    ! $run_file
print(f"Model got rejected {rejected} times")
print(f"Total simulation time: {simulation_time}")
print(f"Total adjoint source calculation time: {adjoint_source_time}")
print(f"Total interpolation time: {interpolation_time}")
print(f"Total source/receiver cutting time: {cutting_time}")


0


 

Current iteration: it0022_model_TrRadius_0.001440 
 



Salvus version 0.0.1-1305-g80e675d9
Floating point size: 32
Compiled for GLL orders: 4, 1
Running on 16 rank(s).

--> Server: 'https://l.mondaic.com/licensing_server', User: 'soelvi', Group: 'SWP_Test'.
--> Negotiating 16 license instance(s) for 'SalvusWaveAlpha' [license version 0.0.1] for 3600 seconds ...
--> Success! [Total duration: 0.51 seconds]
Job started Thu Feb 28 11:57:28 2019

No source specified. Set sources with either --cmd-line-source or --source-toml.
Initializing problem.
Time step set to: 0.0747401 s.
Begin time loop.
Time loop completed in 0.0496451 seconds.
Begin post processing.
Problem complete.

Job finished Thu Feb 28 11:57:29 2019


--> Releasing license early ...
--> Success! [Total duration: 0.52 seconds]
Interpolating velocities for iteration: it0022_model_TrRadius_0.001440, source: src_00
(20736, 2)
Interpolating velocities for iteration: it0022_model_TrRadius_0.001440, source: src_01
(20736, 2

/home/solvi/miniconda3/envs/lasif/lib/python3.6/site-packages/pyasdf/asdf_data_set.py:52: UserWarning: Could not find an element which this points fits into. Maybe you should add some tolerance. Will return the first searched element
  closure_warn(self, *args, **kwargs)


Interpolating velocities for iteration: it0022_model_TrRadius_0.001440, source: src_02
(20736, 2)
Interpolating velocities for iteration: it0022_model_TrRadius_0.001440, source: src_10
(20736, 2)
Interpolating velocities for iteration: it0022_model_TrRadius_0.001440, source: src_11
(20736, 2)
Interpolating velocities for iteration: it0022_model_TrRadius_0.001440, source: src_12
(20736, 2)
Interpolating velocities for iteration: it0022_model_TrRadius_0.001440, source: src_20
(20736, 2)
Interpolating velocities for iteration: it0022_model_TrRadius_0.001440, source: src_21
(20736, 2)
Interpolating velocities for iteration: it0022_model_TrRadius_0.001440, source: src_22
(20736, 2)

Salvus version 0.0.1-1305-g80e675d9
Floating point size: 32
Compiled for GLL orders: 4, 1
Running on 16 rank(s).

--> Server: 'https://l.mondaic.com/licensing_server', User: 'soelvi', Group: 'SWP_Test'.
--> Negotiating 16 license instance(s) for 'SalvusWaveAlpha' [license version 0.0.1] for 3600 seconds ...
--> 

Time loop progress...                                       [99.29%]Time loop completed in 1.39568 seconds.
Begin post processing.
Problem complete.

Job finished Thu Feb 28 11:58:07 2019


--> Releasing license early ...
--> Success! [Total duration: 0.79 seconds]

Salvus version 0.0.1-1305-g80e675d9
Floating point size: 32
Compiled for GLL orders: 4, 1
Running on 16 rank(s).

--> Server: 'https://l.mondaic.com/licensing_server', User: 'soelvi', Group: 'SWP_Test'.
--> Negotiating 16 license instance(s) for 'SalvusWaveAlpha' [license version 0.0.1] for 3600 seconds ...
--> Success! [Total duration: 0.33 seconds]
Job started Thu Feb 28 11:58:08 2019

Less than 7 damping boundary layers specified. This may produce stronger boundary reflections than simple Clayton-Enquist type boundaries. We have found that 7+ or 8+ layers damps reflections quite well. Proceed with caution.
Initializing problem.
Timestep set by user as: 0.05 s.
Automated timestep would be set to: 0.0350832 s.
Begin time l

/home/solvi/miniconda3/envs/lasif/lib/python3.6/site-packages/numpy/core/numerictypes.py:743: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  FutureWarning, stacklevel=2
/home/solvi/miniconda3/envs/lasif/lib/python3.6/site-packages/pyasdf/asdf_data_set.py:52: SalvusMisfitWarning: The amplitude difference between data and synthetic is fairly large.
  closure_warn(self, *args, **kwargs)


...

--> Success! [Total duration: 0.54 seconds]
Job started Thu Feb 28 13:10:41 2019

method name = lbfgs
method type = trust-region
method mode = interactive
Inversion parameter found: MU
Inversion parameter found: LAMBDA
Read initial model from /mnt/md0/Drive/Users/solvi/workspace/synthetic_inversion/smoothiesem2D/nlat02/MESHES/INITIAL/cartesian_2d.e
No bounds detected. Solving unconstrained problem.
mType trust-region
mMode interactive
mTomlDir /Drive/Users/solvi/workspace/synthetic_inversion/smoothiesem2D/nlat02/SALVUSOPT/BACKUP
mTaskFile task.toml
mModelBounds -0.1 < abs(model) < 0.1
initial_max_relative_model_perturbation 0.02
mLbfgsMaxHistory 5
Continuing inversion...
Current iteration: 32
Processing task file done
predicted reduction : -8.08352e-10 	actual reduction : -0.00418644

Model accepted!
Issued task 133. iteration 32 compute gradient for accepted trial model

Job finished Thu Feb 28 13:10:41 2019


--> Releasing license early ...
--> Success! [Total duration: 0.47 seconds]

Timestep set by user as: 0.05 s.
Automated timestep would be set to: 0.0331033 s.
Begin time loop.
Time loop progress...                                       [99.29%]Time loop completed in 0.780084 seconds.
Begin post processing.
Problem complete.

Job finished Thu Feb 28 13:10:58 2019


--> Releasing license early ...
--> Success! [Total duration: 0.44 seconds]

Salvus version 0.0.1-1305-g80e675d9
Floating point size: 32
Compiled for GLL orders: 4, 1
Running on 16 rank(s).

--> Server: 'https://l.mondaic.com/licensing_server', User: 'soelvi', Group: 'SWP_Test'.
--> Negotiating 16 license instance(s) for 'SalvusWaveAlpha' [license version 0.0.1] for 3600 seconds ...
--> Success! [Total duration: 0.57 seconds]
Job started Thu Feb 28 13:10:58 2019

Less than 7 damping boundary layers specified. This may produce stronger boundary reflections than simple Clayton-Enquist type boundaries. We have found that 7+ or 8+ layers damps reflections quite well. Proceed with caution.
Initializing pro


Current iteration: it0033_model_TrRadius_0.000023 
 



Salvus version 0.0.1-1305-g80e675d9
Floating point size: 32
Compiled for GLL orders: 4, 1
Running on 16 rank(s).

--> Server: 'https://l.mondaic.com/licensing_server', User: 'soelvi', Group: 'SWP_Test'.
--> Negotiating 16 license instance(s) for 'SalvusWaveAlpha' [license version 0.0.1] for 3600 seconds ...
--> Success! [Total duration: 0.40 seconds]
Job started Thu Feb 28 13:11:37 2019

No source specified. Set sources with either --cmd-line-source or --source-toml.
Initializing problem.
Time step set to: 0.0730463 s.
Begin time loop.
Time loop completed in 0.044103 seconds.
Begin post processing.
Problem complete.

Job finished Thu Feb 28 13:11:38 2019


--> Releasing license early ...
--> Success! [Total duration: 0.64 seconds]
Interpolating velocities for iteration: it0033_model_TrRadius_0.000023, source: src_00
(20736, 2)
Interpolating velocities for iteration: it0033_model_TrRadius_0.000023, source: src_01
(20736, 2)
Interp

Time loop progress...                                       [99.29%]Time loop completed in 0.896069 seconds.
Begin post processing.
Problem complete.

Job finished Thu Feb 28 13:12:07 2019


--> Releasing license early ...
--> Success! [Total duration: 0.44 seconds]

Salvus version 0.0.1-1305-g80e675d9
Floating point size: 32
Compiled for GLL orders: 4, 1
Running on 16 rank(s).

--> Server: 'https://l.mondaic.com/licensing_server', User: 'soelvi', Group: 'SWP_Test'.
--> Negotiating 16 license instance(s) for 'SalvusWaveAlpha' [license version 0.0.1] for 3600 seconds ...
--> Success! [Total duration: 0.54 seconds]
Job started Thu Feb 28 13:12:08 2019

Less than 7 damping boundary layers specified. This may produce stronger boundary reflections than simple Clayton-Enquist type boundaries. We have found that 7+ or 8+ layers damps reflections quite well. Proceed with caution.
Initializing problem.
Timestep set by user as: 0.05 s.
Automated timestep would be set to: 0.0331033 s.
Begin time 

--> Success! [Total duration: 0.42 seconds]

Salvus version 0.0.1-1305-g80e675d9
Floating point size: 32
Compiled for GLL orders: 4, 1
Running on 16 rank(s).

--> Server: 'https://l.mondaic.com/licensing_server', User: 'soelvi', Group: 'SWP_Test'.
--> Negotiating 16 license instance(s) for 'SalvusWaveAlpha' [license version 0.0.1] for 3600 seconds ...
--> Success! [Total duration: 0.37 seconds]
Job started Thu Feb 28 13:14:14 2019

Less than 7 damping boundary layers specified. This may produce stronger boundary reflections than simple Clayton-Enquist type boundaries. We have found that 7+ or 8+ layers damps reflections quite well. Proceed with caution.
Initializing problem.
Timestep set by user as: 0.05 s.
Automated timestep would be set to: 0.0354092 s.
Begin time loop.
Time loop progress...                                       [99.29%]Time loop completed in 0.822117 seconds.
Begin post processing.
Problem complete.

Job finished Thu Feb 28 13:14:15 2019


--> Releasing license earl